In [1]:
import torch
from gcn import GCN
from utils import load_data, preprocess, normalize_adj_tensor, accuracy, get_train_val_test
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
from metattack import MetaApprox, Metattack
import seaborn as sns
import gc

In [10]:
# Fixed parameters for Cora dataset
seed = 15
epochs = 200
lr = 0.01
hidden = 64  # Changed from 16 to 64 as requested
dataset = 'cora'
model_variant = 'Meta-Self'  # Options: 'A-Meta-Self', 'Meta-Self'
ptb_rates = [0.10]  # Multiple perturbation rates

In [3]:
# Set seeds for reproducibility
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device != 'cpu':
    torch.cuda.manual_seed(seed)

# Load Cora dataset
adj, features, labels = load_data(dataset=dataset)
nclass = max(labels) + 1

Loading cora dataset...
reading cora...
Selecting 1 largest connected components


In [4]:
# Split dataset
val_size = 0.1
test_size = 0.8
train_size = 1 - test_size - val_size
idx = np.arange(adj.shape[0])
idx_train, idx_val, idx_test = get_train_val_test(idx, train_size, val_size, test_size, stratify=labels)
idx_unlabeled = np.union1d(idx_val, idx_test)

# Preprocess without normalizing adjacency yet
adj, features, labels = preprocess(adj, features, labels, preprocess_adj=False)

# Move data to device
if device != 'cpu':
    adj = adj.to(device)
    features = features.to(device)
    labels = labels.to(device)

In [5]:
def train_gcn(adj):
    ''' Train GCN on the given adjacency matrix '''
    # Normalize adjacency matrix for GCN
    norm_adj = normalize_adj_tensor(adj)
    
    # Initialize GCN model - the provided GCN is already 2-layer
    # We just need to set the hidden dimension to 64
    gcn = GCN(nfeat=features.shape[1],
              nhid=hidden,
              nclass=labels.max().item() + 1,
              dropout=0.5)
    
    if device != 'cpu':
        gcn = gcn.to(device)
    
    optimizer = optim.Adam(gcn.parameters(),
                           lr=lr, weight_decay=5e-4)
    
    # Train GCN
    gcn.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        output = gcn(features, norm_adj)
        loss_train = F.nll_loss(output[idx_train], labels[idx_train])
        loss_train.backward()
        optimizer.step()
    
    return gcn

In [6]:
def evaluate(gcn, adj, idx):
    ''' Evaluate GCN on the given adjacency matrix and index '''
    # Normalize adjacency matrix for evaluation
    norm_adj = normalize_adj_tensor(adj)
    
    gcn.eval()
    with torch.no_grad():
        output = gcn(features, norm_adj)
        loss = F.nll_loss(output[idx], labels[idx])
        acc = accuracy(output[idx], labels[idx])
    
    return acc.item()

In [7]:
def run_attack():
    results = {}
    
    print('=== Training GCN on original(clean) graph ===')
    # First, train the GCN on the clean graph
    trained_gcn = train_gcn(adj)
    
    # Set up attack model parameters
    if 'Self' in model_variant:
        lambda_ = 0
    if 'Train' in model_variant:
        lambda_ = 1
    if 'Both' in model_variant:
        lambda_ = 0.5
    
    # Configure attack model
    if 'A' in model_variant:
        model_class = MetaApprox
    else:
        model_class = Metattack
    
    # Evaluate across different perturbation rates
    for ptb_rate in ptb_rates:
        print(f'\n=== Testing perturbation rate: {ptb_rate*100:.1f}% ===')
        perturbations = int(ptb_rate * (adj.sum()//2))
        
        print('=== Setting up attack model ===')
        model = model_class(nfeat=features.shape[1], hidden_sizes=[hidden],
                        nnodes=adj.shape[0], nclass=nclass, dropout=0.5,
                        train_iters=0, attack_features=False, lambda_=lambda_, device=device)
        
        if device != 'cpu':
            model = model.to(device)
            
        # Generate adversarial adjacency matrix focusing on test nodes
        print(f'=== Perturbing graph with {perturbations} edge modifications ===')
        
        modified_adj = model(features, adj, labels, idx_train, idx_test, perturbations, ll_constraint=False)
        modified_adj = modified_adj.detach()
        
        runs = 3  # Reduced from 10 to 3 for quicker execution
        clean_acc = []
        attacked_acc = []
        
        print('=== Evaluating GCN performance ===')
        # Test the already trained GCN on both clean and perturbed graph
        for i in range(runs):
            # Reset the GCN for each run
            trained_gcn = train_gcn(adj)
            
            # Evaluate on clean test data
            clean_acc.append(evaluate(trained_gcn, adj, idx_test))
            
            # Evaluate on perturbed test data (evasion)
            attacked_acc.append(evaluate(trained_gcn, modified_adj, idx_test))
            
            print(f"Run {i+1}/{runs}: Clean acc = {clean_acc[-1]:.4f}, Attacked acc = {attacked_acc[-1]:.4f}")
        
        # Calculate effectiveness metrics
        clean_mean = np.mean(clean_acc)
        clean_std = np.std(clean_acc)
        attack_mean = np.mean(attacked_acc)
        attack_std = np.std(attacked_acc)
        acc_drop = clean_mean - attack_mean
        relative_drop = (acc_drop / clean_mean) * 100
        
        # Print summary statistics
        print(f"\n=== Attack Effectiveness Summary (Perturbation rate: {ptb_rate*100:.1f}%) ===")
        print(f"Clean accuracy: {clean_mean:.4f} ± {clean_std:.4f}")
        print(f"Attacked accuracy: {attack_mean:.4f} ± {attack_std:.4f}")
        print(f"Absolute accuracy drop: {acc_drop:.4f}")
        print(f"Relative accuracy drop: {relative_drop:.2f}%")
        print(f"Effectiveness ratio: {acc_drop/ptb_rate:.4f} (drop per perturbation unit)")
        
        # Assessment
        if acc_drop > 0.10:
            print("Attack assessment: Highly effective")
        elif acc_drop > 0.05:
            print("Attack assessment: Moderately effective")
        elif acc_drop > 0.02:
            print("Attack assessment: Slightly effective")
        else:
            print("Attack assessment: Minimally effective")
        
        # Store results
        results[ptb_rate] = {
            'modified_adj': modified_adj,
            'clean_acc': clean_acc,
            'attacked_acc': attacked_acc,
            'accuracy_drop': acc_drop,
            'relative_drop': relative_drop,
            'effectiveness_ratio': acc_drop/ptb_rate
        }
        
        # Free up memory
        del model
        torch.cuda.empty_cache()
        gc.collect()
    
    # Add overall results
    results['clean_adj'] = adj
    
    # Create and save comparative visualization
    plt.figure(figsize=(10, 6))
    
    ptb_values = list(ptb_rates)
    acc_drops = [results[ptb]['accuracy_drop'] for ptb in ptb_rates]
    rel_drops = [results[ptb]['relative_drop'] for ptb in ptb_rates]
    
    plt.subplot(1, 2, 1)
    plt.plot(ptb_values, acc_drops, 'o-', linewidth=2)
    plt.xlabel('Perturbation Rate')
    plt.ylabel('Absolute Accuracy Drop')
    plt.title('Impact of Perturbation Rate on Accuracy Drop')
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(ptb_values, rel_drops, 'o-', linewidth=2, color='orange')
    plt.xlabel('Perturbation Rate')
    plt.ylabel('Relative Accuracy Drop (%)')
    plt.title('Impact of Perturbation Rate on Relative Accuracy Drop')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('perturbation_impact.png')
    plt.close()
    
    return results

In [8]:
def clean_memory():
    """Clean GPU memory and garbage collect"""
    if device != 'cpu':
        torch.cuda.empty_cache()
    gc.collect()

In [11]:
if __name__ == '__main__':
    torch.cuda.empty_cache()
    results = run_attack()
    
    # Print comparative summary
    print("\n=== Comparative Analysis ===")
    print("Perturbation Rate | Accuracy Drop | Relative Drop | Effectiveness Ratio")
    print("-" * 65)
    for ptb_rate in ptb_rates:
        print(f"{ptb_rate*100:15.1f}% | {results[ptb_rate]['accuracy_drop']:12.4f} | {results[ptb_rate]['relative_drop']:12.2f}% | {results[ptb_rate]['effectiveness_ratio']:18.4f}")

=== Training GCN on original(clean) graph ===

=== Testing perturbation rate: 10.0% ===
=== Setting up attack model ===
=== Perturbing graph with 506 edge modifications ===
=== training surrogate model to predict unlabled data for self-training


Perturbing graph:   1%|▋                                                                | 5/506 [00:00<00:32, 15.34it/s]

GCN loss on unlabled data: 1.9653756618499756
GCN acc on unlabled data: 0.11468812877263583
attack loss: 1.959200143814087
GCN loss on unlabled data: 1.9145087003707886
GCN acc on unlabled data: 0.23390342052313887
attack loss: 1.921037197113037
GCN loss on unlabled data: 2.036731719970703
GCN acc on unlabled data: 0.09557344064386318
attack loss: 2.0375356674194336
GCN loss on unlabled data: 1.9675946235656738
GCN acc on unlabled data: 0.11971830985915494
attack loss: 1.9713417291641235
GCN loss on unlabled data: 1.9677941799163818
GCN acc on unlabled data: 0.14185110663983905
attack loss: 1.9697504043579102


Perturbing graph:   3%|█▋                                                              | 13/506 [00:00<00:18, 27.29it/s]

GCN loss on unlabled data: 2.014777898788452
GCN acc on unlabled data: 0.09909456740442657
attack loss: 2.016979694366455
GCN loss on unlabled data: 1.9977443218231201
GCN acc on unlabled data: 0.09255533199195172
attack loss: 2.0023021697998047
GCN loss on unlabled data: 1.988807201385498
GCN acc on unlabled data: 0.12977867203219318
attack loss: 1.9786558151245117
GCN loss on unlabled data: 2.0361576080322266
GCN acc on unlabled data: 0.11167002012072436
attack loss: 2.031130790710449
GCN loss on unlabled data: 1.91390860080719
GCN acc on unlabled data: 0.16247484909456741
attack loss: 1.9164974689483643
GCN loss on unlabled data: 1.9748203754425049
GCN acc on unlabled data: 0.15492957746478875
attack loss: 1.9723113775253296
GCN loss on unlabled data: 1.9249509572982788
GCN acc on unlabled data: 0.24245472837022136
attack loss: 1.9280815124511719
GCN loss on unlabled data: 1.9430078268051147
GCN acc on unlabled data: 0.16096579476861167
attack loss: 1.9456772804260254


...

GCN loss on unlabled data: 1.987200140953064
GCN acc on unlabled data: 0.10110663983903422
attack loss: 1.9830561876296997
GCN loss on unlabled data: 1.9507243633270264
GCN acc on unlabled data: 0.17655935613682094
attack loss: 1.9494225978851318
GCN loss on unlabled data: 1.9885737895965576
GCN acc on unlabled data: 0.14134808853118713
attack loss: 1.9861088991165161
GCN loss on unlabled data: 1.934905767440796
GCN acc on unlabled data: 0.17404426559356137
attack loss: 1.9410136938095093
GCN loss on unlabled data: 1.9467220306396484
GCN acc on unlabled data: 0.153420523138833
attack loss: 1.9400438070297241
GCN loss on unlabled data: 1.9748859405517578
GCN acc on unlabled data: 0.11720321931589538
attack loss: 1.9765361547470093
GCN loss on unlabled data: 1.9568612575531006
GCN acc on unlabled data: 0.16297786720321933
attack loss: 1.9585978984832764
GCN loss on unlabled data: 1.9808539152145386
GCN acc on unlabled data: 0.09959758551307848
attack loss: 1.9900853633880615


Perturbing graph:  87%|██████████████████████████████████████████████████████▌        | 438/506 [00:11<00:01, 39.01it/s]

GCN loss on unlabled data: 2.0225203037261963
GCN acc on unlabled data: 0.1091549295774648
attack loss: 2.0225629806518555
GCN loss on unlabled data: 2.0039186477661133
GCN acc on unlabled data: 0.10764587525150907
attack loss: 2.007582187652588
GCN loss on unlabled data: 1.959538459777832
GCN acc on unlabled data: 0.15040241448692154
attack loss: 1.9580185413360596
GCN loss on unlabled data: 1.9630845785140991
GCN acc on unlabled data: 0.11569416498993965
attack loss: 1.9618827104568481
GCN loss on unlabled data: 1.9959893226623535
GCN acc on unlabled data: 0.1262575452716298
attack loss: 2.003873109817505
GCN loss on unlabled data: 1.9835032224655151
GCN acc on unlabled data: 0.15845070422535212
attack loss: 1.9873305559158325


Perturbing graph:  87%|██████████████████████████████████████████████████████▋        | 439/506 [00:12<00:01, 36.00it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 31.74 GiB total capacity; 31.27 GiB already allocated; 21.12 MiB free; 31.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF